In [62]:
import pandas as pd

import re
import itertools
from collections import Counter
from typing import  Dict, List, Tuple

from tqdm import tqdm
from transformers import AutoTokenizer

In [4]:
Train_csv_file = '/opt/ml/dataset/train/train.csv'
df = pd.read_csv(Train_csv_file)
df = df.drop_duplicates(['sentence','subject_entity','object_entity','label'])


MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

vocab = list(tokenizer.get_vocab().keys())
unused_list = [word for word in vocab if word.startswith('[unused')]        
print(f'unused token count: {len(unused_list)}')

/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


unused token count: 500


In [5]:
# # UNK_token_string
# unk_idx = tokenizer.get_vocab()[tokenizer.unk_token]
# UNK_word_list = [word for sen in tqdm(df['sentence']) for word in sen.split() if tokenizer.encode(word)]
# UNK_word_counter = Counter(UNK_word_list)

In [6]:
from IPython.core.display import HTML
def word_highligt_html(txt, word, color='black', highlight=None, attr=None):
    if isinstance(word, str):
        txt = txt.replace(word, f'<span style="color: {color}; background-color:{highlight}">{word}</span>')
    else:
        if not isinstance(color, list):
            color = [color] * len(word)
        if not isinstance(highlight, list):
            highlight = [highlight] * len(word)
        for w, c, h in zip(word, color, highlight):
            txt = txt.replace(w, f'<span style="color: {c}; background-color:{h}">{w}</span>')
    return txt

In [7]:
import re
from load_data import load_data
train_dataset = load_data("/opt/ml/dataset/train/train.csv")

txt = ''
count = 1
label = "per:place_of_death"
label_df = train_dataset[train_dataset['label']==label]
for idx, row in label_df.iterrows():
    sent = row['sentence']
    subj = re.sub("'", '', row['subject_entity'].strip())
    obj = re.sub("'", '', row['object_entity'].strip())
    txt += word_highligt_html(sent, [subj, obj], ['white', 'white'],  ['#96C4ED', '#B19CD9']) + '<br/><br/>'
    if count > 2:
        break
    count += 1
HTML(txt)

# 🛰️ UNK 분포

In [8]:
def UNK_word(df_col:pd.DataFrame, unk_idx:int) -> List[Tuple] :
    UNK_word_idx = []
    for i, word in enumerate(tqdm(df_col)) :
        if unk_idx in tokenizer.encode(word) :
            UNK_word_idx.append((i,word))
    return UNK_word_idx

In [9]:
unk_idx = tokenizer.get_vocab()[tokenizer.unk_token] # 3

sub_col = df['subject_entity'].apply(eval).str['word']
sub_UNK = UNK_word(sub_col, unk_idx)

obj_col = df['object_entity'].apply(eval).str['word']
obj_UNK = UNK_word(obj_col, unk_idx)

sen_col = df['sentence']
sen_UNK = UNK_word(sen_col, unk_idx)

100%|██████████| 32428/32428 [00:08<00:00, 3995.40it/s]


In [10]:
print(len(sub_UNK), len(obj_UNK), len(sen_UNK))
print(sub_UNK[0]) # (df row index, entity)
print(obj_UNK[0]) # (df row index, entity)
print(sen_UNK[0]) # (df row index, senetence)

223 282 3217
(21, '스티브 바라캇')
(191, '少弐氏')
(7, '박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.')


## 🛸 한 단어로 이루어지지 않은 entity
- 여러 단어
- 특정 패턴이 존재하는
- 맞추기 어려울 것 같은
+ 위 내용을 만족하는 entity는 entity token wrap 해주는게 어떠려나?

In [174]:
for obj in df['object_entity'] :
    d_obj = eval(obj)
    if d_obj['type'] == 'DAT' :
        if len(d_obj['word'].split()) > 1 :
            print(obj)
            test_encoced = tokenizer.encode(d_obj['word'])
            print(test_encoced)
            Id_to_tokens = tokenizer.convert_ids_to_tokens(test_encoced[1:-1])
            print(Id_to_tokens)
            break

{'word': '1937년 4월 29일', 'start_idx': 9, 'end_idx': 20, 'type': 'DAT'}
[0, 20533, 2440, 24, 2429, 4346, 2210, 2]
['1937', '##년', '4', '##월', '29', '##일']


# 🛰️UNK으로 변하는 word 및 char 확인

In [110]:
test_encoced = tokenizer.encode(obj_UNK[13][1])

print(test_encoced)
Id_to_tokens = tokenizer.convert_ids_to_tokens(test_encoced[1:-1])
Known_char = []
for word in Id_to_tokens :
    if word != tokenizer.unk_token :
        string = word.replace('#','')
        Known_char.append(string)

미겔 후안 세바스티안 피녜라 에체니케
[0, 1107, 2101, 25345, 1269, 30669, 2747, 2283, 3, 1421, 2093, 2209, 2247, 2]


In [ ]:
def Convert_tokenizer_text(text:str) -> List[str]: ## text => tokenizer ids => tokens
    ID_encoced = tokenizer.encode(text) ## text to ids
    ID_to_tokens = tokenizer.convert_ids_to_tokens(ID_encoced[1:-1]) ## ids to token
    return ID_to_tokens

def subword_comparison(wordpiece:List) -> List[str]: ## subword # 제거용
    Known_char = []
    for subword in wordpiece :
        if subword == tokenizer.unk_token :
            Known_char.append(tokenizer.unk_token)
        else :
            string = subword.replace('#','')
            Known_char.extend(string)
    return Known_char


def UNK_word_and_chr(text:str) -> Tuple[List[str], List[str]]:
    word_UNK_list = []
    sub_word_UNK_list = []
    
    wordpieces_tokens = Convert_tokenizer_text(text)

    words_list = text.split()
    tokens_list = tokenizer.convert_tokens_to_string(wordpieces_tokens).split()
    for word, token in zip(words_list, tokens_list) :
        if word != token :
            UNK_ID_encoded = tokenizer.encode(token)[1:-1]
            if UNK_ID_encoded == [unk_idx] :
                word_UNK_list.append(word)
            else :
                subwordpieces_ID_encoded = Convert_tokenizer_text(word)
                Known_subword = subword_comparison(subwordpieces_ID_encoded)
                for sub_char, NK_char in zip(word, Known_subword) :
                    if sub_char != NK_char :
                        sub_word_UNK_list.append(sub_char)
    return word_UNK_list, sub_word_UNK_list

In [84]:
txt = ''
count = 1
for idx, sen in sen_UNK :
    merged_UNK = list(itertools.chain(*UNK_word_and_chr(sen)))
    UNK_len = len(merged_UNK)
    txt += word_highligt_html(sen, merged_UNK, ['white']*UNK_len,  ['#96C4ED']*UNK_len) + '<br/><br/>'
    if count > 50:
        break
    count += 1
HTML(txt)